In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
#download and load dataset
mnist_dataset, mnist_info = tfds.load("mnist",with_info=True,as_supervised=True)

In [3]:
mnist_info.description

'The MNIST database of handwritten digits.'

In [4]:
mnist_info.splits

{'train': <SplitInfo num_examples=60000, num_shards=1>,
 'test': <SplitInfo num_examples=10000, num_shards=1>}

In [5]:
#creating validation sample size and test samples size
num_validation_sample = 0.1 * mnist_info.splits["train"].num_examples
num_validation_sample = tf.cast(num_validation_sample,tf.int64)

num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [6]:
mnist_info.features

FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
})

In [7]:
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255
    return image,label

In [8]:
mnist_dataset.items()

dict_items([('train', <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>), ('test', <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)])

In [9]:
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

train_and_validation = mnist_train.map(scale)
test = mnist_test.map(scale)

In [ ]:
#model

Buffersize = 10000
shuffled_train_and_validation = train_and_validation.shuffle(Buffersize) #shuffling data
train_data = shuffled_train_and_validation.skip(num_validation_sample)      #spliting train
validation_data = shuffled_train_and_validation.take(num_validation_sample) #splitting validation

Batchsize = 100
train_data = train_data.batch(Batchsize)
validation_data = validation_data.batch(Batchsize)

validation_inputs, validation_targets = next(iter(validation_data))

input_size = 784
output_size = 10
hidden_layer_size = 28*2

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
    tf.keras.layers.Dense(output_size,activation="softmax")
])

#optimiser
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

#training
epochs = 5
model.fit(train_data,epochs = epochs, validation_data = (validation_inputs, validation_targets), verbose=2)



Epoch 1/5


d:\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


540/540 - 2s - 3ms/step - accuracy: 0.8893 - loss: 0.3899 - val_accuracy: 0.9500 - val_loss: 0.1471
Epoch 2/5
540/540 - 1s - 1ms/step - accuracy: 0.9500 - loss: 0.1740 - val_accuracy: 0.9500 - val_loss: 0.1094
Epoch 3/5
540/540 - 1s - 1ms/step - accuracy: 0.9605 - loss: 0.1326 - val_accuracy: 0.9700 - val_loss: 0.1117
Epoch 4/5
540/540 - 1s - 1ms/step - accuracy: 0.9681 - loss: 0.1074 - val_accuracy: 0.9600 - val_loss: 0.1341
Epoch 5/5
540/540 - 1s - 1ms/step - accuracy: 0.9734 - loss: 0.0896 - val_accuracy: 0.9700 - val_loss: 0.0863


In [22]:
#testing the model
#after model has seen test values changing the model will cause overfitting
test = test.batch(num_test_samples)
test_loss, test_accuracy = model.evaluate(test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.9669 - loss: 0.1078


In [ ]:
#tweeking the model for higher accuracy
"""
Buffersize = 5000
shuffled_train_and_validation = train_and_validation.shuffle(Buffersize) #shuffling data
train_data = shuffled_train_and_validation.skip(num_validation_sample)      #spliting train
validation_data = shuffled_train_and_validation.take(num_validation_sample) #splitting validation

Batchsize = 250
train_data = train_data.batch(Batchsize)
validation_data = validation_data.batch(Batchsize)

validation_inputs, validation_targets = next(iter(validation_data))

input_size = 784
output_size = 10
hidden_layer_size = 28*28*2

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
    tf.keras.layers.Dense(output_size,activation="softmax")
])

#optimiser
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

#training
epochs = 10
model.fit(train_data,epochs = epochs, validation_data = (validation_inputs, validation_targets), verbose=2)
"""

Epoch 1/10


d:\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


216/216 - 7s - 35ms/step - accuracy: 0.9273 - loss: 0.2325 - val_accuracy: 0.9760 - val_loss: 0.0957
Epoch 2/10
216/216 - 6s - 30ms/step - accuracy: 0.9738 - loss: 0.0852 - val_accuracy: 0.9720 - val_loss: 0.1156
Epoch 3/10
216/216 - 6s - 29ms/step - accuracy: 0.9811 - loss: 0.0612 - val_accuracy: 0.9680 - val_loss: 0.1528
Epoch 4/10
216/216 - 6s - 29ms/step - accuracy: 0.9845 - loss: 0.0511 - val_accuracy: 0.9960 - val_loss: 0.0236
Epoch 5/10
216/216 - 6s - 29ms/step - accuracy: 0.9881 - loss: 0.0372 - val_accuracy: 0.9880 - val_loss: 0.0386
Epoch 6/10
216/216 - 6s - 29ms/step - accuracy: 0.9899 - loss: 0.0329 - val_accuracy: 0.9840 - val_loss: 0.0815
Epoch 7/10
216/216 - 6s - 29ms/step - accuracy: 0.9905 - loss: 0.0307 - val_accuracy: 0.9800 - val_loss: 0.0833
Epoch 8/10
216/216 - 6s - 29ms/step - accuracy: 0.9928 - loss: 0.0242 - val_accuracy: 0.9800 - val_loss: 0.0939
Epoch 9/10
216/216 - 6s - 30ms/step - accuracy: 0.9933 - loss: 0.0232 - val_accuracy: 0.9840 - val_loss: 0.0761
Epo

In [11]:
#testing model after tweeking 
test = test.batch(num_test_samples)
test_loss, test_accuracy = model.evaluate(test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - accuracy: 0.9822 - loss: 0.0709
